In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from shapely.geometry import  Point
import geopandas as gpd
import geoplot as gplt

In [ ]:
# 여러 output 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# **1. DATA LOAD**

In [ ]:
train = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')
test.head()
# train에서 Category, Descript, Resolution 변수가 없음.

In [ ]:
sub = pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')
sub.head()
# class 39개

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
sub.shape

In [ ]:
train.info() #.dtypes

# **2. DATA EDA**

In [ ]:
train["Dates"] = pd.to_datetime(train["Dates"])
train["YEAR"] = train["Dates"].dt.year
train["MONTH"] = train["Dates"].dt.month
train["DAY"] = train["Dates"].dt.day
train["HOUR"] = train["Dates"].dt.hour

test["Dates"] = pd.to_datetime(test["Dates"])
test["YEAR"] = test["Dates"].dt.year
test["MONTH"] = test["Dates"].dt.month
test["DAY"] = test["Dates"].dt.day
test["HOUR"] = test["Dates"].dt.hour

In [ ]:
train.info()
#train.dtypes
# 변수 타입이 object인 것은  수치/범주형으로 변경

In [ ]:
train = train.astype({'Category' : 'category', 
                      'DayOfWeek' : 'string', 
                      'PdDistrict': 'string',
                      'Resolution' : 'string',
                      'Address' : 'string'})
train.dtypes

In [ ]:
test = test.astype({'DayOfWeek' : 'string', 
                    'PdDistrict': 'string',
                    'Address' : 'string'})
test.dtypes

**2-1 target variable**

In [ ]:
# target variable
import matplotlib.pyplot as plt
# import seaborn as sns

TARGET = train['Category']
DF_ = TARGET.value_counts()
len_x = np.arange(len(DF_))

plt.figure(figsize = (20, 12))
plt.bar(len_x, DF_)
plt.title("distribution of target variable(categorical)")
plt.ylabel('count')
plt.xlabel('Category')
plt.xticks(range(0, len(DF_.index)), DF_.index, rotation = "vertical")

# TRIAN_SUM_BY_CATE = train.groupby('Category')
# TRIAN_SUM_BY_CATE.count()
# plt.bar(3, DF_)

In [ ]:
DF_.values
DF_

**2-2 X variables**

In [ ]:
train.isnull().sum()

In [ ]:
aa = train.groupby('Category').count().iloc[:, 0].sort_values(ascending = False) # same as DF_
aa

In [ ]:
# aa.index
# np.delete(aa.index, 1) # 'OTHER OFFENSES'를 제거.

# np.append(np.delete(aa.index, 1), 'OTHER OFFENSES') # 끝에 OTHER OFFENSES를 붙임.
# aa.reindex(np.append(np.delete(aa.index, 1), 'OTHER OFFENSES')) # 다시 재정렬한 index로 aa를 다시 정렬. (필요없는 OTHER OFFENSES를 제일 끝에 배치하기 위함인 듯.)

In [ ]:
aa = train.groupby('Address').count().iloc[:, 0].sort_values(ascending = False) # same as DF_


In [ ]:
aa.head(50)

In [ ]:
np.delete(np.array([[1, 2, 3],[4, 5, 6]]), 2)

# np.delete(1array, 1) : 1차원 array에서 두 번째 원소를 제거 : 2차원 array를 넣으면 1차원 array로 출력됨.
# 반면, np.delete(2array, 1,axis = 0 or 1) : 2차원 array에서 axis = 0이면 두 번째 행을 제거, axis = 1이면 두 번째 열을 제거

In [ ]:
# 위도 경도 plot
# plt.figure(figsize = (20, 12))
# plt.scatter(train['X'], train['Y'])

In [ ]:
# 위도경도 outlier 찾기
train.describe()

In [ ]:
# train2 : outlier 제거한 train set
non_outlier_index = train[train['Y'] != 90].index # 위도,경도가 북극해인 것들.

train2 = train.iloc[non_outlier_index, :]

In [ ]:
train['PdDistrict'].value_counts()

In [ ]:
# plt.figure(figsize = (20, 12))
# # plt.scatter(train2['X'], train2['Y'], c = train2['X'])

# sns.scatterplot(x = 'X', y = 'Y', data = train2, hue = 'PdDistrict')

In [ ]:
# plt.figure(figsize = (20, 12))
# sns.scatterplot(x = 'X', y = 'Y', data = train2,
#                hue = 'Resolution')
# 주황색 NONE이 많음.

In [ ]:
train2['Resolution'].value_counts()

In [ ]:
resolution_not_none_index = train2[train2['Resolution'] != 'NONE'].index

In [ ]:
len(resolution_not_none_index)
resolution_not_none_index

In [ ]:
# none과 그렇지 않은 것을 따로 그림.
# train2_tmp = train2.loc[resolution_not_none_index,:]

# plt.figure(figsize = (20, 12))
# sns.scatterplot(x = 'X', y = 'Y', data = train2_tmp,
#                hue = 'Resolution')

In [ ]:
# plt.figure(figsize = (20, 12))
# g = sns.FacetGrid(train2_tmp, row = "Resolution", hue = "PdDistrict")
# g.map(sns.scatterplot, 'X', 'Y')

In [ ]:
train.info()

In [ ]:
train['Date'] = train.Dates.dt.date

https://www.kaggle.com/yannisp/sf-crime-analysis-prediction

**data split**

In [ ]:
# pandas Series, DataFrame 클래스 확인하기. (어떻게 출력되는지.)

a = pd.Series([1,2,3])
b = {"name" : ["min", "ho"],
     "year" : [2014, 2020]}
bb = pd.DataFrame(b)
a
bb
type(a)
type(bb)

In [ ]:
train_x.info()

In [ ]:
train_y.head()

In [ ]:
# train_x.loc[:,'Address'].unique()
# test_x.loc[:,'Address'].unique()

# a = train_x.loc[:,'Address'].unique()
# b = test_x.loc[:,'Address'].unique()

# len(list(set(a).difference(b))) # A-B
# len(list(set(b).difference(a))) # B-A

# # train과 test에 Address가 겹치지 않는 것이 많다. (train에 있는데 test에 없는 Address. 반대도 마찬가지.)


In [ ]:
type(train_x['DayOfWeek'])
type(np.arange(12))

In [ ]:
# train_x[col].values.reshape(-1,1) /// reshape은 Series에는 안먹음.
# .reshape(-1,1) : 1열로 만들고 싶음. 변경된 배열의 '-1'위치의 차원은 원래 배열의 길이와 남은 차원으로부터 추정.

In [ ]:
train_x = train2[['DayOfWeek', 'PdDistrict',  
                 'X', 'Y', 'YEAR', 'MONTH', 'DAY', 'HOUR']]
train_y = train2[['Category']]

test_x = test[['DayOfWeek', 'PdDistrict', 
                 'X', 'Y', 'YEAR', 'MONTH', 'DAY', 'HOUR']]


type(train_x)
train_x.head()

In [ ]:
from sklearn import preprocessing
cate_col = ['DayOfWeek', 'PdDistrict']
col = ['DayOfWeek']

# 1. LabelEncoder
for col in cate_col:
    le = preprocessing.LabelEncoder()
    train_x.loc[:,col] = le.fit_transform(train_x.loc[:,col])
    test_x.loc[:,col] = le.transform(test_x.loc[:,col])

    
    
# 2. OneHotEncoder
col = 'DayOfWeek'

tmp_tr = pd.DataFrame(train_x.loc[ 0:2000, col ].copy())
tmp_ts = pd.DataFrame(train_x.loc[ 0:2000, col ].copy())

oh = preprocessing.OneHotEncoder()
# oh.fit(tmp_tr)
tr_oh = oh.fit_transform(tmp_tr).toarray()
ts_oh = oh.transform(tmp_ts).toarray()


tr_oh2 = pd.DataFrame(tr_oh, columns = oh.get_feature_names())

tmp_tr
tr_oh2
[tmp_tr,tr_oh2]
pd.concat([tmp_tr,tr_oh2], axis = 1)
pd.concat([tmp_tr,tr_oh2], axis = 1).drop([col], axis = 1)

In [ ]:
# one hot encoder 이 코드 방식으로 종결.
data_dic = {'label' : ['Apple', 'Samsung', 'LG', 'Samsung']}
df = pd.DataFrame(data_dic)

oh = preprocessing.OneHotEncoder()
df_oh = oh.fit_transform(df)
df_oh2 = oh.fit_transform(df).toarray()
df_oh3 = oh.fit_transform(df.to_numpy().reshape(-1,1))

df
df.to_numpy()
df.to_numpy().reshape(-1,1)
df.to_numpy().reshape(-1,2)
print(df_oh)
print(df_oh2)

one_df = pd.DataFrame(df_oh2, columns = oh.get_feature_names())
data = pd.concat([df,one_df], axis = 1).drop(['label'], axis = 1)

one_df
data

In [ ]:
tr_oh.shape
ts_oh.shape
tmp_tr
print(tr_oh)
tmp_ts
print(ts_oh)

In [ ]:
pd.DataFrame(['A', 'B', 'A', 'C'])
tmp_tr
tmp_ts

tr_oh.toarray()
ts_oh
ts_oh.shape

In [ ]:
col = ['DayOfWeek']
train_x.loc[:,col]
type(train_x.loc[:,col])


tmp = oh.fit_transform(train_x.loc[:,col].values.reshape(-1,1)).toarray()
type(tmp)

In [ ]:
train_x

In [ ]:
tmp = train_x.copy()
tmp.head()

tmp_dum = pd.get_dummies(tmp, columns = cate_col)
tmp_dum.head(10)

In [ ]:
train_x.dtypes

In [ ]:
train_x.head()

# **3-1 randomforest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(n_estimators = 20, 
                             max_depth = 5, random_state = 0)
rf1 = clf.fit(train_x, train_y)
predictions_rf1 = rf1.predict_proba(test_x)

In [ ]:
predictions_rf1

In [ ]:
predictions_rf1.shape

# **3-2 catboost**

In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(n_estimators = 10, random_state = 0)
cat.fit(train_x, train_y)
predictions_cat1 = cat.predict_proba(test_x)

In [ ]:
predictions_cat1.shape

# **4. submission**

In [ ]:
# np.linspace(0, 38, 39, dtype='int16')

sub_length = le.inverse_transform(np.linspace(0, 38, 39, dtype='int16'))
sub_length

In [ ]:
sub.head()

In [ ]:
sub.iloc[:,1:]

**3-1 rf sub**

In [ ]:
len(predictions_rf1[0])

In [ ]:
sub.iloc[:,1:] = predictions_rf1 # iloc[1] : substracting 2nd row / iloc[1,2] : approaching one element.
# .iloc[:, 1:]는 첫 열을 제외한 sub table.
sub.head()

In [ ]:
# .iloc[position] 
# .loc[label]

# https://kongdols-room.tistory.com/112

In [ ]:
sub.to_csv('sub1.csv', index = False)

**3-2 cat sub**

In [ ]:
len(predictions_cat1[0])

In [ ]:
sub.iloc[:,1:] = predictions_cat1
sub.head()

In [ ]:
sub.to_csv('sub2.csv', index = False)